In [2]:
import polars as pl
import pandas as pd

In [3]:
# Load the data from products.parquet file.
products = pl.read_parquet('products.parquet')

In [4]:
products

product_id,product_name,unit_price
i64,str,f64
1,"""Product A""",10.0
2,"""Product B""",15.0
3,"""Product C""",20.0
4,"""Product D""",25.0
5,"""Product E""",30.0
6,"""Product F""",12.0
7,"""Product G""",18.0
8,"""Product H""",22.0
9,"""Product I""",28.0


In [5]:
# Load the data from sales.parquet file.
sales = pl.read_parquet('sales.parquet')

In [6]:
sales

sale_id,product_id,quantity_sold,sale_date
i64,i64,i64,str
1,10,20,"""2022-08-12"""
2,1,16,"""2022-12-01"""
3,4,4,"""2023-07-15"""
4,7,1,"""2023-01-21"""
5,1,15,"""2023-08-01"""
6,9,1,"""2023-04-09"""
7,4,4,"""2022-12-31"""
8,6,19,"""2022-11-23"""
9,3,19,"""2023-01-10"""


In [7]:
# Merge the data from both files based on the product_id column to create a new dataset.
combined_sales = products.join(sales, on="product_id", how="inner")

In [8]:
# Calculate the total revenue for each sale by multiplying the quantity_sold with the unit_price for each product.
combined_sales = combined_sales.with_columns(pl.col("product_id"), (pl.col("unit_price") * pl.col("quantity_sold")).alias("total_revenue"))

In [9]:
combined_sales

product_id,product_name,unit_price,sale_id,quantity_sold,sale_date,total_revenue
i64,str,f64,i64,i64,str,f64
10,"""Product J""",35.0,1,20,"""2022-08-12""",700.0
1,"""Product A""",10.0,2,16,"""2022-12-01""",160.0
4,"""Product D""",25.0,3,4,"""2023-07-15""",100.0
7,"""Product G""",18.0,4,1,"""2023-01-21""",18.0
1,"""Product A""",10.0,5,15,"""2023-08-01""",150.0
9,"""Product I""",28.0,6,1,"""2023-04-09""",28.0
4,"""Product D""",25.0,7,4,"""2022-12-31""",100.0
6,"""Product F""",12.0,8,19,"""2022-11-23""",228.0
3,"""Product C""",20.0,9,19,"""2023-01-10""",380.0


In [10]:
combined_sales.columns

['product_id',
 'product_name',
 'unit_price',
 'sale_id',
 'quantity_sold',
 'sale_date',
 'total_revenue']

In [11]:
# Save the transformed data into a new PARQUET file named combined_sales.parquet.
combined_sales = combined_sales.select(pl.col(["sale_id", "product_id", "product_name", "quantity_sold", "unit_price", "sale_date", "total_revenue"]))
combined_sales.write_parquet("combined_sales.parquet")

In [12]:
df = pl.read_parquet('combined_sales.parquet')

In [13]:
df.columns

['sale_id',
 'product_id',
 'product_name',
 'quantity_sold',
 'unit_price',
 'sale_date',
 'total_revenue']

In [14]:
import sqlite3
conn = sqlite3.connect("cheil_sales.db")
cur = conn.cursor()
cur.execute("CREATE TABLE combined_sales(sale_id, product_id, product_name, quantity_sold, unit_price, sale_date, total_revenue)")

In [15]:
combined_sales.to_pandas().to_sql('combined_sales', conn, index=True, if_exists='replace')

50